In [ ]:
"""
20. 4 CRUD методы ORM
Создадим api для работы со складом товаров и разберем возможные реализации CRUD методов в ORM.
Давайте начнем с описания бизнес-задачи. Допустим, у нас есть склад товаров и нам нужно реализовать интерфейс для работы кладовщика.
К нам могут завезти новый товар, какой-то товар может быть больше не актуален для нашего склада, либо же он просто закончился,
и мы ждем новой поставки. 
Реализацию UI данного сервиса мы опустим, так как сейчас нас больше интересует работа с базой данных. 
Нам нужно описать роуты, которые будет дергать клиент для изменения данных в таблице товаров.
Создадим минимальное flask приложение и опишем таблицу товаров:
"""
    
from sqlalchemy import Column, Integer, String, Float, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base
from flask import Flask, jsonify, abort, request
from typing import Dict, Any
 
app = Flask(__name__)
engine = create_engine('sqlite:///sqlite_python.db')
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()
 
 
class Product(Base):
   __tablename__ = 'products'
 
   id = Column(Integer, primary_key=True)
   title = Column(String(200), nullable=False)
   count = Column(Integer, default=0)
   price = Column(Float, default=0)
 
   def __repr__(self):
       return f"Товар {self.title}, в количестве: {self.count}, цена: " \
              f"{self.price}"
 
   def to_json(self) -> Dict[str, Any]:
       return {c.name: getattr(self, c.name) for c in
               self.__table__.columns}
@app.before_request
def before_request_func():
   Base.metadata.create_all(engine)
 
 
if __name__ == '__main__':
   app.run()
 
"""
У каждого товара есть название, количество и цена. Сейчас нам будет этого достаточно.
Стоит обратить внимание на метод to_json().
Здесь мы сериализуем экземпляр продукта в словарь, для последующей передачи в ответе сервера.
"""

In [ ]:
"""Итак, давайте начнем описать наши api методы.
Первый метод на получение списка товаров на складе.
Данный метод должен возвращать список строк, аналогично команде  SELECT FROM TABLE; """

@app.route('/products', methods=['GET'])
def get_all_products():
   '''Получение списка товаров на складе'''
   products = session.query(Product).all()
   products_list = []
   for product in products:
       product_as_dict = product.to_json()
       products_list.append(product_as_dict)
   return jsonify(products_list=products_list), 200

"""
В данном роуте мы получаем список всех товаров и сериализуем их в json формат, возвращая лист объектов.
С точки зрения работы с ORM - здесь все просто, метод all() мы рассматривали на предыдущем уроке.

Следующий роут будет на получение конкретного товара по его ID. 
"""
 
@app.route('/product/<int:id>', methods=['GET'])
def get_product(id):
   '''Получение товара по id'''
   product = session.query(Product).filter(Product.id == id).one_or_none()
   if product is None:
       abort(404)
   return jsonify(product=product.to_json()), 200

"""
Данный роут принимает на вход идентификатор продукта и
с помощью метода запроса filter() получает из базы данных объект с соответствующим ID.
Далее мы его так же сериализуем и отдаем клиенту.

Третий роут будет принимать POST метод для создания нового товара.
В теле запроса мы будем ожидать атрибуты товара и записывать его в базу данных.
"""
 
@app.route('/products', methods=['POST'])
def add_product():
   '''Добавить новый товар на склад'''
   title = request.form.get('title', type=str)
   count = request.form.get('count', type=int)
   price = request.form.get('price', type=float)
   new_product = Product(title=title, count=count, price=price)
   session.add(new_product)
   session.commit()
   return 'Товар успешно создан', 201

"""
Здесь давайте разберем подробнее.
Получив обязательные атрибуты ORM объекта, мы создаем новый экземпляр продукта, передав в конструктор полученные атрибуты. 
Атрибут id, который не определен в __init__, все равно существует из-за того, что колонка id существует в объекте таблицы продукта.
Стандартно mapper() создает атрибуты класса для всех колонок, что есть в объекте Table.
Эти атрибуты существуют как Python дескрипторы и определяют его функциональность.
Так как мы не сказали SQLAlchemy сохранить товар в базу, его id выставлено на None.
Когда позже мы сохраним его, в этом атрибуте будет храниться сформированное значение. 
Метод add() объекта Session добавляет вновь созданный экземпляр товара в сессию.
Его состояние будет сохранено в базе данных при следующем коммите. Для коммита как раз используется session.commit(),
который очищает ожидающие изменения и фиксирует текущую транзакцию в базе. После этого товар будет успешно занесен в базу.

Следующий роут нам нужен для изменения товара.
При покупке товара количество товара на складе изменится, у него может поменять цена или название.
"""
app.route('/product/<int:id>', methods=['PATCH'])
def update_product(id):
   '''Обновить товар на складе'''
   title = request.form.get('title', type=str)
   count = request.form.get('count', type=int)
   price = request.form.get('price', type=float)
   product = session.query(Product).filter(Product.id == id).one_or_none()
   if title:
       product.title = title
   if count:
       product.count = count
   if price:
       product.price = price
   session.commit()
   # # 2 вариант UPDATE
   # session.query(Product).filter(Product.id == id)\
   #     .update({Product.title:title,
   #              Product.count:count,
   #              Product.price:price})
   ## 3 вариант UPDATE
   # from sqlalchemy import update
   # query = update(Product).where(Product.id == id).values(title=title,
   #                                                        count=count,
   #                                                        price=price)
   # print(query)
   # session.execute(query)
   return 'Товар успешно обновлен', 200
 
"""
Мы используем метод PATCH, который частично изменяет ресурс.
Получаем входные атрибуты продукта, по ID запрашивает экземпляр продукта и обновляем измененные атрибуты. Далее также комитим изменения.
Но UPDATE строки мы могли сделать и иначе. С помощью метода update() объекта Query, либо с помощью объекта  Update. 
Сперва разберем метод update() для нашего запроса, который соответствует комментированному коду  - 2 вариант UPDATE. Очевидно, что данный метод вызывает sql - команду UPDATE для обновления данных.
Чуть подробнее стоит остановиться на 3 варианте это обновление данных с помощью объекта Update.
Данный объект представляет конструкцию обновления и вызывает с помощью функции update() из пакета SqlAlchemy.
Конструкция формирует SQL - запрос, который нужно передать на выполнение в метод сессии execute().
Попробуйте вывести результат функции update() в терминал - вы увидите классический SQL запрос.
Реализация через метод объекта Query актуальна и для удаления из базы данных.
А рассмотренный нами 3 вариант реализации через специальную конструкцию аналогично можно использовать для методов INSERT и DELETE,
заменив функцию update() на insert() и delete() соответственно.
В нашем случае 2 и 3 варианты реализации нам не подходят, так как могут возникнуть ошибки,
когда какой-либо из атрибутов  продукт не придет.
Но в случае, когда все атрибуты являются обязательными входными параметрами,то обе реализации более чем актуальны. 

Остался еще один роут - роут на удаление товара.
"""
@app.route('/product/<int:id>', methods=['DELETE'])
def delete_product(id):
   '''Убрать товар со склада'''
   # с помощью метода объекта Query
   session.query(Product).filter(Product.id == id).delete()
   session.commit()
   # # используя специальную конструкцию delete
   # from sqlalchemy import delete
   # query = delete(Product).where(Product.id == id)
   # session.execute(query)
   return 'Товар успешно удален', 200
"""
Здесь, мы получаем ID товара и удаляем его из базы данных. 
В целом наша API готова к использованию. Можем создать несколько товаров, изменить, удалить товары и просмотреть список всех товаров.
"""